# M2 - 220 - Experiment #2 - Joint-Labelling with noisy dataset

Experiment on reference dataset

Ankit Agrawal, Sarsij Tripathi, Manu Vardhan, Vikas Sihag, Gaurav Choudhary, et Nicola Dragoni. 2022. « BERT-Based Transfer-Learning Approach for Nested Named-Entity Recognition Using Joint Labeling ». Applied Sciences 12 (3): 976. https://doi.org/10.3390/app12030976.

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
print(os.environ["CUDA_VISIBLE_DEVICES"])

3


In [2]:
import torch
torch.cuda.empty_cache()
torch.cuda.is_available()

True

In [3]:
#!pip install --upgrade transformers datasets spacy transformers[sentencepiece] seqeval

## Initialisation
Set the BASE path.
If run on Google Colab, will also mout Google Drive to the moutpoint given below

In [4]:
import os, sys
from pathlib import Path

ENV_IS_GOOGLE_COLAB = True if 'google.colab' in str(get_ipython()) else False
os.environ["ENV_IS_GOOGLE_COLAB"] = str(ENV_IS_GOOGLE_COLAB)

if ENV_IS_GOOGLE_COLAB:
  from google.colab import drive
  mountpoint = Path("/content/drive")
  drive.mount(str(mountpoint)) # Mount gdrive to BASE
  base = mountpoint / "MyDrive/article_icdar_2023" # Adapt this to your situation
  sys.path.append(str(base)) # Add BASE to Python Path
  BASE = Path(base).resolve() # Make BASE absolute
  DATASETS =  BASE / "dataset"
else:
  BASE = Path(os.path.dirname(os.path.realpath("__file__"))).resolve() # If not on GColab, BASE will be the directory of this notebook
  DATASETS = Path('/work/stual/dataset_ICDAR')
  OUT_BASE = Path('/work/stual/res_ICDAR/method_2')

print(sys.path)
print(BASE)
print(DATASETS)
print(OUT_BASE)

['/lrde/home2/stual/stage_DAS/m2_joint-labelling_for_ner', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages']
/lrde/home2/stual/stage_DAS/m2_joint-labelling_for_ner
/work/stual/dataset_ICDAR
/work/stual/res_ICDAR/method_2


## Constants

**Don't forget to check tokenizer name in *model_util_IO.py* and *model_util_IOB2.py* files (same name as model) !**

In [5]:
RUN_CAMEMBERT_IO = True
RUN_CAMEMBERT_IOB2 = False
#Can't run together because of convert_tokenizer_
RUN_PTRN_CAMEMBERT_IO = False
RUN_PTRN_CAMEMBERT_IOB2 = False

# Number of times a model will be trained & evaluated on each a dataset
N_RUNS = 10

## Parameters

In [6]:
# COMMON CONSTANTS

TRAINING_CONFIG = {
    "evaluation_strategy": "steps",
    "eval_steps": 100,
    "max_steps": 5000,
    "learning_rate": 1e-4,
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size": 16,
    "weight_decay": 1e-5,
    "load_best_model_at_end": True,
    "greater_is_better":True,
    "metric_for_best_model": "f1",
    "save_strategy": "steps",
    "save_steps": 100,
    "save_total_limit": 1
}

In [7]:
import pandas as pd
from datasets import load_from_disk

#Print examples from datasets
def loadExample(INPUT_DIR,set_length:int,i:int,subset:str):
    set_ = load_from_disk(INPUT_DIR / f"huggingface_{set_length}")
    data = {"tokens": set_[subset][i]["tokens"],
            "labels": set_[subset][i]["ner_tags"]}
    df = pd.DataFrame.from_dict(data)
    print(df)

# 221 - Train & eval : IO Pero OCR dataset with CamemBERT model

In [8]:
MODEL_NAME = "dalembert"#"camembert_ner"
MODEL = "pjox/dalembert"#"Jean-Baptiste/camembert-ner"
LABEL = "io"
FOLDER = "221-camembert-ner-joint-labelling-io"

### 221.1 Load IO dataset

In [9]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
INPUT_DIR = OUT_BASE / f"m2-experiment_2_prepared_dataset_pero_ocr_{LABEL}_{MODEL_NAME}"
METRICS_OUTPUT_DIR = OUT_BASE / "m2-220-experiment_2_metrics"
INPUT_DIR, METRICS_OUTPUT_DIR

(PosixPath('/work/stual/res_ICDAR/method_2/m2-experiment_2_prepared_dataset_pero_ocr_io_dalembert'),
 PosixPath('/work/stual/res_ICDAR/method_2/m2-220-experiment_2_metrics'))

### Example

In [10]:
loadExample(INPUT_DIR,TRAINSETS_SIZES[-1],10,"train")

   tokens             labels
0     ĠDu            I-PER+O
1      ff            I-PER+O
2      au            I-PER+O
3       -            I-PER+O
4       P            I-PER+O
5      au            I-PER+O
6   illac            I-PER+O
7      Ġ(            I-PER+O
8      Ch            I-PER+O
9       s            I-PER+O
10     ),                O+O
11    ĠEn       I-SPAT+i_LOC
12     gh       I-SPAT+i_LOC
13    ien       I-SPAT+i_LOC
14      ,           I-SPAT+O
15     âĢ           I-SPAT+O
16      ©           I-SPAT+O
17     16  I-SPAT+i_CARDINAL
18      .                O+O


/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


### 221.2 Fine-tuning with IO labels - train & eval

In [11]:
from config import logger
from datasets import load_from_disk
import json
from camembert_utils.util_IO import init_model, train_eval_loop, _convert_tokenizer

def train_bert(metrics_output_directory):
    # Train & evaluate loop
    for run in range(1, N_RUNS + 1):
        output_dir = metrics_output_directory / f"run_{run}"
        output_dir.mkdir(exist_ok=True)

        for trainset_size in TRAINSETS_SIZES:
            datasetdir = INPUT_DIR / f"huggingface_{trainset_size}"
            logger.info(f"Running on datasets in {datasetdir}")
            logger.info(f"Metrics will be saved in {output_dir}")
            
            model, tokenizer, training_args = init_model(MODEL, local_config)
            logger.info(f"{model} #{run}, will save in {output_dir}")

            train_dev_test = load_from_disk(datasetdir)
            train = train_dev_test["train"]
            dev = train_dev_test["dev"]
            test = train_dev_test["test"]
            metrics = train_eval_loop(model,         # Implicit. Must be setbefore calling train_bert()
                                      training_args, # Idem
                                      tokenizer,
                                      train,dev,test)

            # Save the dev and test metrics
            metrics_file = output_dir / f"test_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[0], o)

            metrics_file = output_dir / f"dev_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[1], o)
                
            torch.cuda.empty_cache()

2023-04-28 10:58:27.108818: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 10:58:28.166510: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-28 10:58:28.166624: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-28 10:58:28.166635: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [12]:
import time
import datetime

if RUN_CAMEMBERT_IO:
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH

    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning model for IO labels")

TypeError: init_model() missing 1 required positional argument: 'num_run'

# 222 - Train & eval : IOB2 Pero OCR dataset with CamemBERT model

In [13]:
MODEL_NAME = "camembert_ner"
MODEL = "Jean-Baptiste/camembert-ner"
LABEL = "iob2"
FOLDER = "222-camembert-ner-joint-labelling-iob2"

### 222.1 Load IOB2 dataset

In [14]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
INPUT_DIR = OUT_BASE / f"m2-experiment_2_prepared_dataset_pero_ocr_{LABEL}_{MODEL_NAME}"
METRICS_OUTPUT_DIR = OUT_BASE / "m2-220-experiment_2_metrics"
INPUT_DIR, METRICS_OUTPUT_DIR

(PosixPath('/work/stual/res_ICDAR/method_2/m2-experiment_2_prepared_dataset_pero_ocr_iob2_camembert_ner'),
 PosixPath('/work/stual/res_ICDAR/method_2/m2-220-experiment_2_metrics'))

### Example

In [15]:
loadExample(INPUT_DIR,TRAINSETS_SIZES[-1],10,"train")

            tokens               labels
0  Duffau-Pauillac            I-b_PER+O
1                (            I-i_PER+O
2              Chs            I-i_PER+O
3                )            I-i_PER+O
4                ,                  O+O
5          Enghien       I-b_SPAT+b_LOC
6                ,           I-i_SPAT+O
7               16  I-i_SPAT+b_CARDINAL
8                .                  O+O


/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


### 222.2 Fine-tuning with IOB2 labels - train & eval

In [16]:
from config import logger
from datasets import load_from_disk
import json
from camembert_utils.util_IOB2 import init_model, train_eval_loop, _convert_tokenizer

def train_bert(metrics_output_directory):
    # Train & evaluate loop
    for run in range(1, N_RUNS + 1):
        output_dir = metrics_output_directory / f"run_{run}"
        output_dir.mkdir(exist_ok=True)

        for trainset_size in TRAINSETS_SIZES:
            datasetdir = INPUT_DIR / f"huggingface_{trainset_size}"
            logger.info(f"Running on datasets in {datasetdir}")
            logger.info(f"Metrics will be saved in {output_dir}")
            
            model, tokenizer, training_args = init_model(MODEL, local_config,run)
            logger.info(f"{model} #{run}, will save in {output_dir}")
            
            train_dev_test = load_from_disk(datasetdir)
            train = train_dev_test["train"]
            dev = train_dev_test["dev"]
            test = train_dev_test["test"]
            metrics = train_eval_loop(model,         # Implicit. Must be setbefore calling train_bert()
                                      training_args, # Idem
                                      tokenizer,
                                      train,dev,test)

            # Save the dev and test metrics
            metrics_file = output_dir / f"test_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[0], o)

            metrics_file = output_dir / f"dev_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[1], o)
                
            torch.cuda.empty_cache()

[nltk_data] Downloading package punkt to
[nltk_data]     /lrde/home2/stual/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
import time
import datetime

if RUN_CAMEMBERT_IOB2:
    print(_convert_tokenizer.name_or_path)
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH

    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning model for IOB2 labels")

Skipped finetuning model for IOB2 labels


# 223 - Train & eval : IO Pero OCR dataset with Pretrained CamemBERT model

In [18]:
MODEL_NAME = "pretrained_camembert_ner"
MODEL = "HueyNemud/das22-10-camembert_pretrained"
LABEL = "io"
FOLDER = "223-pretrained-camembert-ner-joint-labelling-io"

### 223.1 Load IO dataset

In [19]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
INPUT_DIR = OUT_BASE / f"m2-experiment_2_prepared_dataset_pero_ocr_{LABEL}_{MODEL_NAME}"
METRICS_OUTPUT_DIR = OUT_BASE / "m2-220-experiment_2_metrics"
INPUT_DIR, METRICS_OUTPUT_DIR

(PosixPath('/work/stual/res_ICDAR/method_2/m2-experiment_2_prepared_dataset_pero_ocr_io_pretrained_camembert_ner'),
 PosixPath('/work/stual/res_ICDAR/method_2/m2-220-experiment_2_metrics'))

In [20]:
loadExample(INPUT_DIR,TRAINSETS_SIZES[-1],10,"train")

            tokens             labels
0  Duffau-Pauillac            I-PER+O
1                (            I-PER+O
2              Chs            I-PER+O
3                )            I-PER+O
4                ,                O+O
5          Enghien       I-SPAT+i_LOC
6                ,           I-SPAT+O
7               16  I-SPAT+i_CARDINAL
8                .                O+O


/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


### 223.2 Fine-tuning with IO labels - train & eval

In [21]:
from config import logger
from datasets import load_from_disk
import json
from camembert_utils.util_IO import init_model, train_eval_loop, _convert_tokenizer

def train_bert(metrics_output_directory):
    # Train & evaluate loop
    for run in range(1, N_RUNS + 1):
        output_dir = metrics_output_directory / f"run_{run}"
        output_dir.mkdir(exist_ok=True)

        for trainset_size in TRAINSETS_SIZES:
            datasetdir = INPUT_DIR / f"huggingface_{trainset_size}"
            logger.info(f"Running on datasets in {datasetdir}")
            logger.info(f"Metrics will be saved in {output_dir}")
            
            model, tokenizer, training_args = init_model(MODEL, local_config,run)
            logger.info(f"{model} #{run}, will save in {output_dir}")

            train_dev_test = load_from_disk(datasetdir)
            train = train_dev_test["train"]
            dev = train_dev_test["dev"]
            test = train_dev_test["test"]
            metrics = train_eval_loop(model,         # Implicit. Must be setbefore calling train_bert()
                                      training_args, # Idem
                                      tokenizer,
                                      train,dev,test)

            # Save the dev and test metrics
            metrics_file = output_dir / f"test_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[0], o)

            metrics_file = output_dir / f"dev_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[1], o)
                
            torch.cuda.empty_cache()

In [22]:
import time
import datetime

if RUN_PTRN_CAMEMBERT_IO:
    print(_convert_tokenizer.name_or_path)
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH


    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning model for IO labels")

HueyNemud/das22-10-camembert_pretrained


Some weights of the model checkpoint at HueyNemud/das22-10-camembert_pretrained were not used when initializing CamembertForTokenClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at HueyNemud/das22-10-camembert_pretrained and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this mo

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Precision-l1,Recall-l1,F1-l1,Accuracy-l1,Precision-l2,Recall-l2,F1-l2,Accuracy-l2,Precision-all,Recall-all,F1-all,Accuracy-all,Precision-das,Recall-das,F1-das,Accuracy-das,Per,Act,Act L1,Act L2,Desc,Titreh,Titrep,Spat,Loc,Cardinal,Ft,Titre
100,No log,0.386581,0.930543,0.930814,0.930679,0.935586,0.902091,0.928609,0.915158,0.944571,0.945243,0.920053,0.932478,0.966844,0.920514,0.924839,0.922671,0.955707,0.925693,0.932588,0.929128,0.949380,"{'precision': 0.9062049062049062, 'recall': 0.9289940828402367, 'f1': 0.9174579985390797, 'number': 676}","{'precision': 0.8765880217785844, 'recall': 0.930635838150289, 'f1': 0.9028037383177571, 'number': 519}","{'precision': 0.8765880217785844, 'recall': 0.9602385685884692, 'f1': 0.9165085388994307, 'number': 503}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 16}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 30}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 33}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}","{'precision': 0.9202797202797203, 'recall': 0.9481268011527377, 'f1': 0.9339957416607523, 'number': 694}","{'precision': 0.9309895833333334, 'recall': 0.9395532194480947, 'f1': 0.9352517985611511, 'number': 761}","{'precision': 0.9624277456647399, 'recall': 0.9823008849557522, 'f1': 0.9722627737226278, 'number': 678}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}"
200,No log,0.250152,0.937339,0.952326,0.944773,0.952923,0.918182,0.954331,0.935907,0.959757,0.946667,0.946036,0.946351,0.972159,0.930460,0.950675,0.940459,0.965958,0.942040,0.956425,0.949178,0.967097,"{'precision': 0.9462209302325582, 'recall': 0.9630177514792899, 'f1': 0.9545454545454545, 'number': 676}","{'precision': 0.9053803339517625, 'recall': 0.9402697495183044, 'f1': 0.9224952741020794, 'number': 519}","{'precision': 0.9053803339517625, 'recall': 0.9701789264413518, 'f1': 0.9366602687140115, 'number': 503}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 16}","{'precision': 0.21951219512195122, 'recall': 0.3, 'f1': 0.2535211267605634, 'number': 30}","{'precision': 0.7575757575757576, 'recall': 0.7575757575757576, 'f1': 0.7575757575757576, 'number': 33}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}","{'precision': 0.9410112359550562, 'recall': 0.9654178674351584, 'f1': 0.9530583214793741, 'number': 694}","{'precision': 0.9236641221374046, 'recall': 0.9540078843626807, 'f1': 0.938590820943762, 'number': 761}","{'precision': 0.9838235294117647, 'recall': 0.9867256637168141, 'f1': 0.9852724594992636, 'number': 678}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}"
300,No log,0.196779,0.948326,0.954942,0.951622,0.956720,0.935567,0.952756,0.944083,0.964186,0.953722,0.947368,0.950535,0.975323,0.943457,0.950382,0.946907,0.969754,0.955746,0.957169,0.956457,0.971653,"{'precision': 0.9631811487481591, 'recall': 0.9674556213017751, 'f1': 0.9653136531365313, 'number': 676}","{'precision': 0.9436893203883495, 'recall': 0.9364161849710982, 'f1': 0.9400386847195358, 'number': 519}","{'precision': 0.9436893203883495, 'recall': 0.9662027833001988, 'f1': 0.9548133595284873, 'number': 503}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 16}","{'precision': 0.26666666666666666, 'recall': 0.4, 'f1': 0.32, 'number': 30}","{'precision': 0.8181818181818182, 'recall': 0.8181818181818182, 'f1': 0.8181818181818182, 'number': 33}","{'precision': 0.2857142857142857, 'recall': 0.3333333333333333, 'f1': 0.30769230769230765, 'number': 6}","{'precision': 0.9457917261055635, 'recall': 0.9553314121037464, 'f1': 0.9505376344086022, 'number': 694}","{'precision': 0.9401820546163849, 'recall': 0.9500657030223391, 'f1': 0.9450980392156864, 'number': 761}","{'precision': 0.9823529411764705, 'recall': 0.9852507374631269, 'f1': 0.9837997054491899, 'number': 678}","{'precision': 1.0,

The following columns in the evaluation set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 676
  Batch size = 16
/lrde/home2/stual/stage_DAS/m2_joint-labelling_for_ner/camembert_utils/util_IO.py:95: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+O seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Prec

/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.9462209302325582, 'recall': 0.9630177514792899, 'f1': 0.9545454545454545, 'number': 676}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9053803339517625, 'recall': 0.9402697495183044, 'f1': 0.9224952741020794, 'number': 519}" of type <class 'dict'> for key "eval/ACT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9053803339517625, 'recall'

...

Trainer is attempting to log a value of "{'precision': 0.9778761061946902, 'recall': 0.9807692307692307, 'f1': 0.9793205317577548, 'number': 676}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9253731343283582, 'recall': 0.9556840077071291, 'f1': 0.9402843601895735, 'number': 519}" of type <class 'dict'> for key "eval/ACT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.949317738791423, 'recall': 0.9681908548707754, 'f1': 0.9586614173228347, 'number': 503}" of type <class 'dict'> for key "eval/ACT_L1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.391304347826087, 'recall': 0.5625, 'f1': 0.4615

Trainer is attempting to log a value of "{'precision': 0.953185955786736, 'recall': 0.9632063074901446, 'f1': 0.9581699346405229, 'number': 761}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9809384164222874, 'recall': 0.9867256637168141, 'f1': 0.9838235294117647, 'number': 678}" of type <class 'dict'> for key "eval/CARDINAL" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.6666666666666666, 'recall': 0.2857142857142857, 'f1': 0.4, 'number': 7}" of type <class 'dict'> for key "eval/FT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}" of type <class 'dict

/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+O seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
Trainer is attempting to log a value of "{'precision': 0.9533372711163615, 'recall': 0.9578635014836795, 'f1': 0.955595026642984, 'number': 1685}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8696035242290749, 'recall': 0.9021937842778793, 'f1': 0.8855989232839838, 'number': 1094}" of type <class 'dict'> for key "eval/ACT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9392752203721841, 'recall': 0.930164888457808, 'f1': 0.9346978557504872, 'number': 1031}" of type <class 'dict'> for key "eval/ACT_L1" as a

Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}" of type <class 'dict'> for key "eval/TITRE" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Run-time is equal to 0:05:21.794683


# 224 - Train & eval : IOB2 Pero dataset with Pretrained CamemBERT model

In [23]:
MODEL_NAME = "pretrained_camembert_ner"
MODEL = "HueyNemud/das22-10-camembert_pretrained"
LABEL = "iob2"
FOLDER = "224-pretrained-camembert-ner-joint-labelling-iob2"

### 224.1 Load IOB2 dataset

In [24]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
INPUT_DIR = OUT_BASE / f"m2-experiment_2_prepared_dataset_pero_ocr_{LABEL}_{MODEL_NAME}"
METRICS_OUTPUT_DIR = OUT_BASE / "m2-220-experiment_2_metrics"
INPUT_DIR, METRICS_OUTPUT_DIR

(PosixPath('/work/stual/res_ICDAR/method_2/m2-experiment_2_prepared_dataset_pero_ocr_iob2_pretrained_camembert_ner'),
 PosixPath('/work/stual/res_ICDAR/method_2/m2-220-experiment_2_metrics'))

In [25]:
loadExample(INPUT_DIR,TRAINSETS_SIZES[-1],10,"train")

            tokens               labels
0  Duffau-Pauillac            I-b_PER+O
1                (            I-i_PER+O
2              Chs            I-i_PER+O
3                )            I-i_PER+O
4                ,                  O+O
5          Enghien       I-b_SPAT+b_LOC
6                ,           I-i_SPAT+O
7               16  I-i_SPAT+b_CARDINAL
8                .                  O+O


/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


### 224.2 Fine-tuning with IOB2 labels - train & eval

In [26]:
from config import logger
from datasets import load_from_disk
import json
from camembert_utils.util_IOB2 import init_model, train_eval_loop, _convert_tokenizer

torch.cuda.empty_cache()

def train_bert(metrics_output_directory):
    # Train & evaluate loop
    for run in range(1, N_RUNS + 1):
        output_dir = metrics_output_directory / f"run_{run}"
        output_dir.mkdir(exist_ok=True)

        #Fine-tuning on the biggest dataset
        for trainset_size in TRAINSETS_SIZES:
            datasetdir = INPUT_DIR / f"huggingface_{trainset_size}"
            logger.info(f"Running on datasets in {datasetdir}")
            logger.info(f"Metrics will be saved in {output_dir}")
            
            model, tokenizer, training_args = init_model(MODEL, local_config,run)
            logger.info(f"{model} #{run}, will save in {output_dir}")

            train_dev_test = load_from_disk(datasetdir)
            
            train = train_dev_test["train"]
            dev = train_dev_test["dev"]
            test = train_dev_test["test"]
            metrics = train_eval_loop(model,         # Implicit. Must be setbefore calling train_bert()
                                      training_args, # Idem
                                      tokenizer,
                                      train,dev,test)

            # Save the dev and test metrics
            metrics_file = output_dir / f"test_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[0], o)

            metrics_file = output_dir / f"dev_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[1], o)
                
            torch.cuda.empty_cache()

In [27]:
import time
import datetime

if RUN_PTRN_CAMEMBERT_IOB2:
    print(_convert_tokenizer.name_or_path)
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH

    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning model for IOB2 labels")

loading file sentencepiece.bpe.model from cache at /lrde/home2/stual/.cache/huggingface/hub/models--HueyNemud--das22-10-camembert_pretrained/snapshots/a54f5177528f2e319b97b1f3960d0a00fd9e3ef3/sentencepiece.bpe.model
loading file tokenizer.json from cache at /lrde/home2/stual/.cache/huggingface/hub/models--HueyNemud--das22-10-camembert_pretrained/snapshots/a54f5177528f2e319b97b1f3960d0a00fd9e3ef3/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /lrde/home2/stual/.cache/huggingface/hub/models--HueyNemud--das22-10-camembert_pretrained/snapshots/a54f5177528f2e319b97b1f3960d0a00fd9e3ef3/special_tokens_map.json
loading file tokenizer_config.json from cache at /lrde/home2/stual/.cache/huggingface/hub/models--HueyNemud--das22-10-camembert_pretrained/snapshots/a54f5177528f2e319b97b1f3960d0a00fd9e3ef3/tokenizer_config.json
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (fro

HueyNemud/das22-10-camembert_pretrained


loading configuration file config.json from cache at /lrde/home2/stual/.cache/huggingface/hub/models--HueyNemud--das22-10-camembert_pretrained/snapshots/a54f5177528f2e319b97b1f3960d0a00fd9e3ef3/config.json
Model config CamembertConfig {
  "_name_or_path": "HueyNemud/das22-10-camembert_pretrained",
  "architectures": [
    "CamembertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 5,
  "classifier_dropout": null,
  "eos_token_id": 6,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O+O",
    "1": "I-b_PER+O",
    "2": "I-i_PER+O",
    "3": "I-b_PER+b_TITREH",
    "4": "I-i_PER+b_TITREH",
    "5": "I-i_PER+i_TITREH",
    "6": "I-b_ACT+O",
    "7": "I-i_ACT+O",
    "8": "I-b_DESC+O",
    "9": "I-i_DESC+O",
    "10": "I-b_DESC+b_ACT",
    "11": "I-i_DESC+b_ACT",
    "12": "I-i_DESC+i_ACT",
    "13": "I-b_DESC+b_TITREP",
    "14": "I-i_DESC+b_TITREP",
    "15": "I-i_DESC+i_TITRE

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Precision-l1,Recall-l1,F1-l1,Accuracy-l1,Precision-l2,Recall-l2,F1-l2,Accuracy-l2,Precision-das,Recall-das,F1-das,Accuracy-das,Precision-l1l2,Recall-l1l2,F1-l1l2,Accuracy-l1l2,Precision-all,Recall-all,F1-all,Accuracy-all,Per,Act,Act L1,Act L2,Desc,Titreh,Titrep,Spat,Loc,Cardinal,Ft,Titre
100,No log,0.672828,0.929549,0.921773,0.925644,0.927993,0.888720,0.918110,0.903176,0.941660,0.939024,0.922770,0.930826,0.964566,0.914423,0.923277,0.918829,0.948114,0.915493,0.925603,0.920520,0.928626,0.910279,0.920164,0.915195,0.953113,"{'precision': 0.8877840909090909, 'recall': 0.9245562130177515, 'f1': 0.9057971014492754, 'number': 676}","{'precision': 0.8631386861313869, 'recall': 0.9113680154142582, 'f1': 0.8865979381443299, 'number': 519}","{'precision': 0.8631386861313869, 'recall': 0.9403578528827038, 'f1': 0.9000951474785919, 'number': 503}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 16}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 30}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 33}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}","{'precision': 0.909217877094972, 'recall': 0.9380403458213257, 'f1': 0.9234042553191489, 'number': 694}","{'precision': 0.9221938775510204, 'recall': 0.9488188976377953, 'f1': 0.9353169469598965, 'number': 762}","{'precision': 0.958092485549133, 'recall': 0.9778761061946902, 'f1': 0.9678832116788321, 'number': 678}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}"
200,No log,0.381792,0.953462,0.940312,0.946841,0.940648,0.919016,0.941207,0.929979,0.950266,0.956671,0.926099,0.941137,0.964946,0.943615,0.941155,0.942383,0.956973,0.945306,0.939262,0.942274,0.941154,0.935095,0.934547,0.934821,0.957606,"{'precision': 0.9331395348837209, 'recall': 0.9497041420118343, 'f1': 0.9413489736070382, 'number': 676}","{'precision': 0.90625, 'recall': 0.9499036608863198, 'f1': 0.9275634995296331, 'number': 519}","{'precision': 0.90625, 'recall': 0.9801192842942346, 'f1': 0.941738299904489, 'number': 503}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 16}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 30}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 33}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}","{'precision': 0.9228611500701263, 'recall': 0.9481268011527377, 'f1': 0.9353233830845772, 'number': 694}","{'precision': 0.944300518134715, 'recall': 0.9566929133858267, 'f1': 0.9504563233376793, 'number': 762}","{'precision': 0.9706744868035191, 'recall': 0.976401179941003, 'f1': 0.9735294117647059, 'number': 678}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}"
300,No log,0.269880,0.950363,0.947999,0.949179,0.953809,0.930840,0.953806,0.942183,0.962541,0.935762,0.940746,0.938247,0.972665,0.960580,0.962011,0.961295,0.975702,0.938466,0.952921,0.945638,0.954315,0.932987,0.948048,0.940457,0.967603,"{'precision': 0.9689349112426036, 'recall': 0.9689349112426036, 'f1': 0.9689349112426036, 'number': 676}","{'precision': 0.9212007504690432, 'recall': 0.9460500963391136, 'f1': 0.9334600760456274, 'number': 519}","{'precision': 0.9212007504690432, 'recall': 0.9761431411530815, 'f1': 0.9478764478764479, 'number': 503}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 16}","{'precision': 0.2222222222222222, 'recall': 0.26666666666666666, 'f1': 0.2424242424242424, 'number': 30}","{'precision': 0.2765957446808511, 'recall': 0.3939393939393939, 'f1': 0.325, 'number': 33}","{'precision': 0.25, 'recall': 0.16666666666666666, 'f1': 0.2, 'number': 6}","{'precision': 0.9377652050919377, 'recall': 0.9553314121037464, 'f1': 0.9464668094218414, 'number': 694}","{'precision': 0.9430051813471503, 'recall': 0.9553805774278216, 'f1': 0.9491525423728813, 'number': 762}","{'precision': 0.97953216374269, 'recall': 0.9882005899705014, 'f1': 0.983847283406

The following columns in the evaluation set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, ner_tags. If tokens, ner_tags are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 676
  Batch size = 16
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+O seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarnin

...

/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.9896602658788775, 'recall': 0.9911242603550295, 'f1': 0.9903917220990391, 'number': 676}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9257884972170687, 'recall': 0.9614643545279383, 'f1': 0.9432892249527409, 'number': 519}" of type <class 'dict'> for key "eval/ACT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.958984375, 'recall': 0.976

Trainer is attempting to log a value of "{'precision': 0.25, 'recall': 0.3333333333333333, 'f1': 0.28571428571428575, 'number': 6}" of type <class 'dict'> for key "eval/TITREP" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9601139601139601, 'recall': 0.9711815561959655, 'f1': 0.9656160458452723, 'number': 694}" of type <class 'dict'> for key "eval/SPAT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9594771241830066, 'recall': 0.963254593175853, 'f1': 0.9613621480026195, 'number': 762}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9838472834067548, 'recall': 0.9882005899705014, 'f1': 0.98601

/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+O seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O+O seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
Trainer is attempting to log a value of "{'precision': 0.9444119368051492, 'recall': 0.9578635014836795, 'f1': 0.9510901591043016, 'number': 1685}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorr

Trainer is attempting to log a value of "{'precision': 0.9582790091264668, 'recall': 0.9645669291338582, 'f1': 0.961412688031393, 'number': 762}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9824304538799414, 'recall': 0.9896755162241888, 'f1': 0.9860396767083027, 'number': 678}" of type <class 'dict'> for key "eval/CARDINAL" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.6666666666666666, 'recall': 0.2857142857142857, 'f1': 0.4, 'number': 7}" of type <class 'dict'> for key "eval/FT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}" of type <class 'dict

Run-time is equal to 0:06:42.112903
